In [2]:
#-*- coding: utf-8 -*-
import pandas as pd
import numpy as np

In [12]:
a2016m01 = pd.read_csv("a2016m01.csv", encoding = "ISO-8859-1")
a2016m01



,campus,nível,nome do curso,modalidade,turno,alunos matriculados
0,Campus Alegrete,Graduação,Ciência da Computação,Bacharelado,Noturno,151
1,Campus Alegrete,Graduação,Engenharia Agrícola,Bacharelado,Integral,180
2,Campus Alegrete,Graduação,Engenharia Civil,Bacharelado,Integral,283
3,Campus Alegrete,Graduação,Engenharia de Software,Bacharelado,Noturno,147
4,Campus Alegrete,Graduação,Engenharia de Telecomunicações,Bacharelado,Integral,96
5,Campus Alegrete,Graduação,Engenharia Elétrica,Bacharelado,Integral,209
6,Campus Alegrete,Graduação,Engenharia Mecânica,Bacharelado,Integral,227
7,Campus Alegrete,Pós-Graduação,ESPECIALIZAÇÃO EM ENGENHARIA ECONÔMICA,Especialização,Integral,26
8,Campus Alegrete,Pós-Graduação,Mestrado em Engenharia Elétrica,Mestrado,Integral,17
9,Campus Alegrete,Pós-Graduação,Mestrado em Engenharias,Mestrado,Integral,31
